In [4]:
import prepare_vocab
import train
import config
from data.loader import DataLoader
from utils import constant

In [5]:
# %Load glove vectors, get IDs for each token, lookup for each id in embedings and create embeding file for out dataset.
vocab_params = config.VocabParameters()
training_params = config.TrainingParameters()
opt = vars(vocab_params)
opt['num_class'] = len(constant.LABEL_TO_ID)
opt.update(vars(training_params))
vocab = prepare_vocab.prepare_voabulary (vocab_params)

loading files...
2525296 tokens from 68124 examples loaded from ./dataset/tacred/train.json.
802558 tokens from 22631 examples loaded from ./dataset/tacred/dev.json.
539009 tokens from 15509 examples loaded from ./dataset/tacred/test.json.
loading glove...
2196017 words loaded from glove.
building vocab...
vocab built with 21/62152 words.
calculating oov...
train oov: 2525296/2525296 (100.00%)
dev oov: 802558/802558 (100.00%)
test oov: 539009/539009 (100.00%)
building embeddings...
embedding size: 21 x 30
dumping to files...
all done.


In [12]:
import utils.vocab as Vocab
vocab_file = vocab_params.vocab_dir + '/vocab.pkl'
vocab = Vocab(vocab_file, load=True)

TypeError: 'module' object is not callable

In [3]:
# load data
print("Loading data from {} with batch size {}...".format(vocab_params.data_dir, training_params.batch_size))
train_batch = DataLoader(vocab_params.data_dir+ '/train.json', training_params.batch_size, opt, vocab, evaluation=False)
dev_batch = DataLoader(vocab_params.data_dir + '/dev.json', training_params.batch_size, opt, vocab, evaluation=True)


Loading data from ./dataset/tacred with batch size 50...


AttributeError: module 'utils.vocab' has no attribute 'word2id'

In [ ]:
# Train model using Position Aware
train.train_model(vocab_params, training_params, train_batch, dev_batch, model_id='00')

# Train model using LSTM
train.train_model(vocab_params, training_params, train_batch, dev_batch, model_id='01')

1234